# Результаты SRN, обученного на MJSynth + SynthText

In [1]:
import sys  
sys.path.insert(0, '/home/astrashnov/PaddleOCR/tools/')

import pandas as pd
from tqdm.notebook import tqdm
from eval_predictions import general_stat

%cd /home/astrashnov/PaddleOCR

/home/astrashnov/PaddleOCR


In [2]:
imgs_dirs = ('ic13/images', 'ic15/ch4_test_word_images_gt', 'iiit5k/test', 'svt/par', 'svt-perspective/par1', 'cute/images')
gt_paths = ('ic13/gt.txt', 'ic15/test_groundtruth_1811.txt', 'iiit5k/iiit5k_1k_test.txt',
            'svt/gt.txt', 'svt-perspective/gt.txt', 'cute/gt.txt')
names = ('ICDAR 2013', 'ICDAR 2015', 'IIIT5K', 'SVT', 'SVTP', 'CUTE')
sizes = (857, 1811, 3000, 647, 645, 288)

alphabet = "0123456789abcdefghijklmnopqrstuvwxyz"

In [3]:
def get_results(config, checkout, predicts):
    results = {}
    for name, img_dir, gt_path in zip(names, imgs_dirs, gt_paths):
        print(name)
        !python3 tools/infer_rec.py -c $config -o Global.pretrained_model=$checkout Global.load_static_weights=false Global.infer_img=/home/pparamonov/data_clean/$img_dir > /dev/null
        stat = general_stat(predicts, '/home/pparamonov/data_clean/' + gt_path, alphabet=alphabet)
        results[name] = stat
        print()
    df = pd.DataFrame.from_dict(results).transpose()
    acc_columns = ['Accuracy (LC + UC, all words)',
                   'Accuracy (LC + UC, long words)',
                   'Accuracy (MC, all words)',
                   'Accuracy (MC, long words)',]
    all_columns = df.columns
    for column in acc_columns:
        df[column] *= 100
    df = df.transpose()
    df['All'] = 0.0
    for name, size in zip(names, sizes):
        df['All'] += df[name] * size
    df['All'] /= sum(sizes)
    return df.style.format('{:.4f}').format('{:.2f}%', subset=pd.IndexSlice[acc_columns, :])

#### Обученная на датасете без аугументаций:

In [4]:
get_results('configs/rec/srn_MJSST_MC_Num.yml',
            './output/rec/srn_MJSST_MC_Num/iter_epoch_8',
            'output/rec/predicts_srn_MJSST_MC_Num.txt')

ICDAR 2013
/home/astrashnov/anaconda3/envs/paddle_env/lib/python3.7/site-packages/skimage/morphology/_skeletonize.py:241: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  0, 1, 1, 0, 0, 1, 0, 0, 0], dtype=np.bool)
/home/astrashnov/anaconda3/envs/paddle_env/lib/python3.7/site-packages/skimage/morphology/_skeletonize.py:256: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.ht

,ICDAR 2013,ICDAR 2015,IIIT5K,SVT,SVTP,CUTE,All
"Accuracy (LC + UC, all words)",23.34%,76.20%,88.83%,0.00%,0.00%,14.24%,59.13%
"Accuracy (LC + UC, long words)",12.90%,65.62%,73.08%,0.00%,0.00%,7.14%,48.45%
"Accuracy (MC, all words)",94.87%,76.20%,88.83%,87.79%,78.76%,78.12%,84.98%
"Accuracy (MC, long words)",93.55%,65.62%,73.08%,90.70%,85.37%,57.14%,75.67%
"Normalized edit distance 1 (LC + UC, all words)",0.4928,0.1064,0.0409,1.0057,1.0027,0.7659,0.3112
"Normalized edit distance 1 (LC + UC, long words)",0.4068,0.0873,0.0788,1.0000,1.0024,0.7304,0.3100
"Normalized edit distance 1 (MC, all words)",0.0213,0.1064,0.0409,0.0533,0.0978,0.1051,0.0637
"Normalized edit distance 1 (MC, long words)",0.0187,0.0873,0.0788,0.0408,0.0732,0.2218,0.0756
"Normalized edit distance 2 (LC + UC, all words)",0.4797,0.1016,0.0439,1.0040,1.0018,0.8051,0.3110
"Normalized edit distance 2 (LC + UC, long words)",0.4037,0.0889,0.1141,1.0000,1.0024,0.6923,0.3231


#### Обученная на датасете с аугументациями:

In [5]:
get_results('configs/rec/srn_MJSST_aug_MC_Num.yml',
            './output/rec/srn_MJSST_aug_MC_Num/latest',
            'output/rec/predicts_srn_MJSST_aug_MC_Num.txt')

ICDAR 2013
/home/astrashnov/anaconda3/envs/paddle_env/lib/python3.7/site-packages/skimage/morphology/_skeletonize.py:241: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  0, 1, 1, 0, 0, 1, 0, 0, 0], dtype=np.bool)
/home/astrashnov/anaconda3/envs/paddle_env/lib/python3.7/site-packages/skimage/morphology/_skeletonize.py:256: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.ht

,ICDAR 2013,ICDAR 2015,IIIT5K,SVT,SVTP,CUTE,All
"Accuracy (LC + UC, all words)",23.10%,76.48%,90.50%,0.00%,0.00%,12.15%,59.78%
"Accuracy (LC + UC, long words)",12.90%,67.19%,71.79%,0.00%,0.00%,0.00%,48.03%
"Accuracy (MC, all words)",92.07%,76.48%,90.50%,87.48%,77.52%,78.12%,85.26%
"Accuracy (MC, long words)",91.40%,67.19%,71.79%,90.70%,90.24%,35.71%,74.86%
"Normalized edit distance 1 (LC + UC, all words)",0.5001,0.1056,0.0424,1.0029,1.0042,0.8391,0.3153
"Normalized edit distance 1 (LC + UC, long words)",0.4125,0.0631,0.0633,1.0023,1.0000,0.7410,0.2986
"Normalized edit distance 1 (MC, all words)",0.0309,0.1056,0.0424,0.0485,0.0958,0.1563,0.0666
"Normalized edit distance 1 (MC, long words)",0.0209,0.0631,0.0633,0.0249,0.0463,0.2455,0.0605
"Normalized edit distance 2 (LC + UC, all words)",0.4873,0.0970,0.0392,1.0032,1.0037,0.8271,0.3098
"Normalized edit distance 2 (LC + UC, long words)",0.4095,0.0641,0.0908,1.0022,1.0000,0.6864,0.3078


#### На обученной модели из репозитория:

In [6]:
get_results('configs/rec/srn_MJSST_MC_Num.yml',
            './pretrain_models/rec_r50_vd_srn_train/best_accuracy',
            'output/rec/predicts_srn_MJSST_MC_Num.txt')

ICDAR 2013
/home/astrashnov/anaconda3/envs/paddle_env/lib/python3.7/site-packages/skimage/morphology/_skeletonize.py:241: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  0, 1, 1, 0, 0, 1, 0, 0, 0], dtype=np.bool)
/home/astrashnov/anaconda3/envs/paddle_env/lib/python3.7/site-packages/skimage/morphology/_skeletonize.py:256: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.ht

,ICDAR 2013,ICDAR 2015,IIIT5K,SVT,SVTP,CUTE,All
"Accuracy (LC + UC, all words)",23.34%,79.85%,91.10%,0.00%,0.00%,13.54%,60.95%
"Accuracy (LC + UC, long words)",12.90%,73.44%,73.72%,0.00%,0.00%,7.14%,50.67%
"Accuracy (MC, all words)",94.52%,79.85%,91.10%,89.64%,84.03%,86.81%,87.76%
"Accuracy (MC, long words)",94.62%,73.44%,73.72%,90.70%,90.24%,57.14%,78.45%
"Normalized edit distance 1 (LC + UC, all words)",0.4922,0.0903,0.0369,1.0038,1.0046,0.7929,0.3065
"Normalized edit distance 1 (LC + UC, long words)",0.4095,0.0649,0.0683,1.0023,1.0000,0.7137,0.2997
"Normalized edit distance 1 (MC, all words)",0.0199,0.0903,0.0369,0.0404,0.0707,0.0756,0.0531
"Normalized edit distance 1 (MC, long words)",0.0174,0.0649,0.0683,0.0203,0.0390,0.0843,0.0552
"Normalized edit distance 2 (LC + UC, all words)",0.4801,0.0837,0.0362,1.0029,1.0040,0.8064,0.3036
"Normalized edit distance 2 (LC + UC, long words)",0.4046,0.0671,0.0908,1.0022,1.0000,0.6627,0.3070
